In [ ]:
# %pip install instructor pydantic --upgrade openai

# Topics today: 

**1. Get the _kind_ of output you want**  
**2. Minimize Hallucinations with response validators**

By the end of the demo, you'll see some examples of how we can minimize hallucinations and gain more confidence in your model's output.


# "Guarantee" Response Structure

In [ ]:
from typing import List
from pydantic import BaseModel
from openai import OpenAI
import json

client = OpenAI()

class PythonPackage(BaseModel):
    name: str
    author: str

class Packages(BaseModel):
    packages: List[PythonPackage]

class PythonPackage(BaseModel):
    name: str
    author: str

resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Return the `name`, and `author` of pydantic, in a json object."
        },
    ]
)

Packages.model_validate_json(resp.choices[0].message.content)

If everything is fine, we might receive an output similar to 

```python
json.loads({"name": "pydantic", "author": "Samuel Colvin"}).
```

### LLM responses with markdown code blocks

```python
json.loads("""
```json
{
  "name": "pydantic",
  "author": "Samuel Colvin"
}
'''
""")
>>> JSONDecodeError: Expecting value: line 1 column 1 (char 0
```

### LLM responses with prose

```python
json.loads("""
Ok heres the authors of pydantic: Samuel Colvin, and the name this library

{
  "name": "pydantic",
  "author": "Samuel Colvin"
}
""")
>>> JSONDecodeError: Expecting value: line 1 column 1 (char 0
```

The ^ content may contain valid JSON, but it isn't considered valid JSON without understanding the language model's behavior. 

## Calling Tools

By defining the api payload as a Pydantic model, we can leverage the `response_model` argument to instruct the model to generate the desired output. This is a powerful feature that allows us to generate structured data from any language model!

Here's the same example as above without using pydantic's schema generation.

In [6]:

resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Pydantic and FastAPI?",
        },
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "Requirements",
                "description": "A list of packages and their first authors.",
                "parameters": {
                    "$defs": {
                        "Package": {
                            "properties": {
                                "name": {"title": "Name", "type": "string"},
                                "author": {"title": "Author", "type": "string"},
                            },
                            "required": ["name", "author"],
                            "title": "Package",
                            "type": "object",
                        }
                    },
                    "properties": {
                        "packages": {
                            "items": {"$ref": "#/$defs/Package"},
                            "title": "Packages",
                            "type": "array",
                        }
                    },
                    "required": ["packages"],
                    "title": "Packages",
                    "type": "object",
                },
            },
        }
    ],
    tool_choice={
        "type": "function",
        "function": {"name": "Requirements"},
    },
)

resp = json.loads(resp.choices[0].message.tool_calls[0].function.arguments)


Now, notice in this example that the prompts we use contain purely the data we want, where the tools and `tool_choice` now capture the schemas we want to output. This separation of concerns makes it much easier to organize the 'data' and the 'description' of the data that we want back out.

In [8]:
class PythonPackage(BaseModel):
    name: str
    author: str


class Packages(BaseModel):
    packages: List[PythonPackage]


resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": "Pydantic and FastAPI?",
        },
    ],
    tools=[
        {
            "type": "function",
            "function": {
                "name": "Requirements",
                "description": "A list of packages and their first authors.",
                "parameters": Packages.model_json_schema(),
            },
        }
    ],
    tool_choice={
        "type": "function",
        "function": {"name": "Requirements"},
    },
)

Packages.model_validate_json(
    resp.choices[0].message.tool_calls[0].function.arguments
)

Packages(packages=[PythonPackage(name='pydantic', author='Samuel Colvin'), PythonPackage(name='fastapi', author='Sebastian Ramirez')])

```json {
  "packages": [
    {
      "name": "pydantic",
      "author": "Samuel Colvin"
    },
    {
      "name": "fastapi",
      "author": "Sebastián Ramírez"
    }
  ]
}
```

The example we provided above is somewhat contrived, but it illustrates how Pydantic can be utilized to generate structured data from language models. Now, let's employ Instructor to streamline this process. Instructor is a compact library that enhances the OpenAI client by offering convenient features.

# Case Study: Search query segmentation

Let's consider a practical example. Imagine we have a search engine capable of comprehending intricate queries. For instance, if we make a request to find "recent advancements in AI", we could provide the following payload:

```json
{
  "rewritten_query": "novel developments advancements ai artificial intelligence machine learning",
  "published_daterange": {
    "start": "2023-09-17",
    "end": "2021-06-17"
  },
  "domains_allow_list": ["arxiv.org"]
}
```

If we peek under the hood, we can see that the query is actually a complex object, with a date range, and a list of domains to search in. We can model this structured output in Pydantic using the instructor library.

In [9]:
from typing import List
import datetime
from pydantic import BaseModel

class DateRange(BaseModel):
    start: datetime.date
    end: datetime.date

class SearchQuery(BaseModel):
    rewritten_query: str
    published_daterange: DateRange
    domains_allow_list: List[str]

    async def execute():
        # Return the search results of the rewritten query
        return api.search(json=self.model_dump())

This pattern empowers us to restructure the user's query for improved performance, without requiring the user to understand the inner workings of the search backend.

In [ ]:
# %pip install anthropic

In [15]:
import instructor
from openai import OpenAI

# Enables response_model in the openai client
client = instructor.patch(OpenAI())

def search(query: str) -> SearchQuery:
    return client.chat.completions.create(
        model="gpt-4",
        response_model=SearchQuery,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for a search engine. Today's date is {datetime.date.today()}"
            },
            {
                "role": "user",
                "content": query
            }
        ],
    )

search("recent advancements in AI")

SearchQuery(rewritten_query='latest advancements in artificial intelligence', published_daterange=DateRange(start=datetime.date(2023, 4, 4), end=datetime.date(2024, 4, 4)), domains_allow_list=['research.google', 'ai.google', 'www.oaic.gov.au', 'www.ibm.com', 'www.technologyreview.com'])

## Conclusion of Demo 1

By defining the api payload as a Pydantic model, we can leverage the `response_model` argument to instruct the model to generate the desired output. This is a powerful feature that allows us to generate structured data from any language model!

# Demo 2: Minimize LLM Hallucinations

**Goal: You know how to minimize LLM hallucinations using validators**

## Intro to Validators
Validators are functions that take a value, check a property, raise an error, and return a value. They can be used to enforce constraints on model inputs and outputs.

In [16]:
def validation_function(value):
    if condition(value):
        raise ValueError("Value is not valid")
    return mutation(value)

For instance, consider validating a name field. Here’s how you can enforce a space in the name using Annotated and AfterValidator:

In [25]:
from typing_extensions import Annotated
from pydantic import BaseModel, ValidationError, AfterValidator, ValidationInfo

def name_must_contain_space(v: str) -> str:
    if " " not in v:
        raise ValueError("Name must contain a space.")
    return v.lower()

class UserDetail(BaseModel):
    age: int
    name: Annotated[str, AfterValidator(name_must_contain_space)] 

person = UserDetail.model_validate({"age": 24, "name": "Jason"})

ValidationError: 1 validation error for UserDetail
name
  Value error, Name must contain a space. [type=value_error, input_value='Jason', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error

## Context-Driven Validators

Validators can also be used to enforce context-specific constraints. 

Validator context becomes crucial in directing language models. It helps in excluding specific content (like competitor names) or focusing on relevant topics, which is particularly effective in question-answering scenarios.

For instance, consider a validator that checks if a name is in a list of names, and raises an error if it isn't. Enhancing validators with `ValidationInfo` adds nuanced control. For example, removing dynamic stopwords from a text requires us to pass in some context:

In [22]:
def remove_stopwords(v: str, info: ValidationInfo):
    context = info.context
    if context:
        stopwords = context.get('stopwords', set())
        v = ' '.join(w for w in v.split() if w.lower() not in stopwords)
    return v

class Response(BaseModel):
    message: Annotated[str, AfterValidator(remove_stopwords)]

Passing dynamic context to the validator:

In [32]:
class Response(BaseModel):
    message: Annotated[str, AfterValidator(remove_stopwords)]

data = {'message': 'This is an example response'}
print(Response.model_validate(data))  
#> text='This is an example response'

print(Response.model_validate(
    data, context={
        'stopwords': ['this', 'is', 'an'] 
    }))
#> text='example response'

message='This is an example response'
message='example response'


## Validation Using an LLM 

Some rules are easier to express using natural language. For instance, consider the following rule: **'don't say objectionable things'**. This rule is difficult to express using a validator function, but easy to express using natural language. We can use an LLM to generate a validator function from this rule.

Consider this example where we want some light moderation on a question answering model. We want to ensure that the answer does not contain objectionable content. We can use an LLM to generate a validator function that checks if the answer contains objectionable content.

In [35]:
import instructor

from openai import OpenAI
from instructor import llm_validator
from pydantic import BaseModel, BeforeValidator
from typing_extensions import Annotated

client = instructor.patch(OpenAI())

NoEvil = Annotated[
    str,
    BeforeValidator(
        llm_validator("don't say objectionable things", client)
    )]

class QuestionAnswer(BaseModel):
    question: str
    answer: NoEvil

QuestionAnswer.model_validate({
    "question": "What is the meaning of life?",
    "answer": "Sex, drugs, and rock'n roll"
})

ValidationError: 1 validation error for QuestionAnswer
answer
  Assertion failed, The phrase 'Sex, drugs, and rock'n roll' may be considered objectionable due to the mention of drugs and sex. [type=assertion_error, input_value="Sex, drugs, and rock'n roll", input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/assertion_error

```python
ValidationError: 1 validation error for QuestionAnswer
answer
  Assertion failed, The phrase 'Sex, drugs, and rock'n roll' may be considered objectionable due to the mention of drugs and sex. [type=assertion_error, input_value="Sex, drugs, and rock'n roll", input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/assertion_error
```

## Grounding responses in context

Many organizations worry about hallucinations in their llm responses. To address this we can use validators to ensure that the model's responses are grounded in the context used to generate the prompt.

For instance, let's consider a question-answering model that provides answers based on a text chunk. To ensure that the model's response is firmly based on the given text chunk, we can employ a validator. In this case, we can use `ValidationInfo` to verify the response. By using a straightforward validator, we can guarantee that the model's response is firmly grounded in the provided text chunk.

In [39]:
def citation_exists(v: str, info: ValidationInfo):
    context = info.context
    if context:
        context = context.get("text_chunk")
        if v not in context: # (1)!
            raise ValueError(f"Citation `{v}` not found in text")
    return v

Citation = Annotated[str, AfterValidator(citation_exists)]

class AnswerWithCitation(BaseModel):
    answer: str
    citation: Citation

Now lets consider an example where we want to answer a question using a text chunk. We can use a validator to ensure that the model's response is grounded in the provided text chunk.

In [47]:
text_chunk= "please note that currently, Madison now no longer is the capital of Wisconsin; Milwaukee is."

q = "what is the capital of wisconsin?"

AnswerWithCitation.model_validate({
    "answer": "The capital of Wisconsin is Madison",
    "citation": "Madison is the capital."
}, context={"text_chunk": text_chunk})

ValidationError: 1 validation error for AnswerWithCitation
citation
  Value error, Citation `Madison is the capital.` not found in text [type=value_error, input_value='Madison is the capital.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error

Alhought the answer in this example was correct, the validator will raise an error because the citation is not in the text chunk. Which can help us identify and correct the model's 'hallucination' which can not be defined as incorrectly cited information.

We can use OpenAI to generate a response to a question using a text chunk. We can use a validator to ensure that the model's response is grounded in the provided text chunk.

In [49]:
resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=AnswerWithCitation,
    messages=[
        {"role": "user", "content": f"Answer the question `{q}` using the text chunk\n`{text_chunk}`"},
    ],
    validation_context={"text_chunk": text_chunk},
)
resp

AnswerWithCitation(answer='Milwaukee', citation='please note that currently, Madison now no longer is the capital of Wisconsin; Milwaukee is.')

## Conclusion to Demo 2
The power of these techniques lies in the flexibility and precision with which we can use Pydantic to describe and control outputs.

Whether it's moderating content, avoiding specific topics or competitors, or even ensuring responses are grounded in provided context, Pydantic's `BaseModel` offers a very natural way to describe the data structure we want, while validation functions and ValidationInfo provide the flexibility to enforce these constraints.

### References
* [Steering LLMs wih Pydantic](https://blog.pydantic.dev/blog/2024/01/04/steering-large-language-models-with-pydantic/)
* [Pydantic docs](https://docs.pydantic.dev/latest/concepts/validators/)